In [1]:
import tensorflow as tf
import pickle
import numpy as np
import os

CIFAR_DIR = './cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [2]:
def load_data(filename):
    """read data from data file."""
    with open (filename, "rb") as f:
        data = pickle.load(f,encoding='bytes')
        return data[b'data'], data[b'labels']
    
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 -1 # 归一化
        self._labels = np.hstack(all_labels)
        print(self._data.shape, self._labels.shape)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0 ## 遍历起始位置
        if self._need_shuffle:
            self._shuffle_data()
    
    def _shuffle_data(self):
        # [0, 1, 2, 3, 4, 5] -> [5, 3, 2, 4, 0 ,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0 
                end_indicator = batch_size
            else:
                raise Exception("have no more example")
        
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]    
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1,6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

batch_data, batch_labels = train_data.next_batch(10)
print(batch_data, batch_labels)

(50000, 3072) (50000,)
(10000, 3072) (10000,)
[[ 0.08235294  0.05882353 -0.01176471 ...  0.0745098  -0.04313725
  -0.06666667]
 [ 0.67058824  0.63137255  0.6627451  ... -0.23921569 -0.41176471
  -0.17647059]
 [-0.37254902 -0.38039216 -0.52156863 ...  0.74117647  0.74117647
   0.78039216]
 ...
 [-0.45098039 -0.45098039 -0.38823529 ... -0.55294118 -0.67843137
  -0.88235294]
 [ 0.05882353  0.03529412  0.04313725 ...  0.08235294  0.09019608
   0.09019608]
 [-0.85882353 -0.85882353 -0.84313725 ... -0.48235294 -0.41960784
  -0.44313725]] [7 8 5 7 4 4 4 7 0 8]


In [3]:
x = tf.placeholder(tf.float32, [None, 3072])
# [None], eg: [0,6,5,3]
y = tf.placeholder(tf.int64, [None])

# 多层神经网络
hidden1 = tf.layers.dense(x, 100, activation=tf.nn.relu)
hidden2 = tf.layers.dense(hidden1, 100, activation=tf.nn.relu)
hidden3 = tf.layers.dense(hidden2, 50, activation=tf.nn.relu)
y_ = tf.layers.dense(hidden3, 10)
"""
# (3072, 10)
w = tf.get_variable('w', [x.get_shape()[-1], 10],
                   initializer=tf.random_normal_initializer(0, 1))

# (10, )
b = tf.get_variable('b', [10],
                   initializer=tf.constant_initializer(0.0))

# [None, 3072] * [3072, 10] = [None, 10]
y_ = tf.matmul(x, w) + b
"""

# mean square loss
"""
# course: 1 + e^x
# api: e^x / sum(e^x)
# [[0.01, 0.9, ..., 0.03], []]
p_y = tf.nn.softmax(y_)
# 5 -> [0,0,0,0,1,0,0,0,0,0]
y_one_hot = tf.one_hot(y, 10, dtype=tf.float32)
loss = tf.reduce_mean(tf.square(y_one_hot - p_y))
"""

loss = tf.losses.sparse_softmax_cross_entropy(labels=y,logits=y_)
# y_ -> softmax
# y -> one_hot
#loss = ylog_

# indices
predict = tf.argmax(y_, 1)
# [1,0,1,0]
correct_prediction = tf.equal(predict, y)
accuracy= tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

W0712 21:06:10.377247  2656 deprecation.py:323] From <ipython-input-3-e66ebdaae113>:6: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W0712 21:06:10.381367  2656 deprecation.py:506] From D:\anaconda3\envs\deep_learning\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0712 21:06:10.796257  2656 deprecation.py:323] From D:\anaconda3\envs\deep_learning\lib\site-packages\tensorflow\python\ops\losses\losses_impl.py:121: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadca

In [4]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps =100

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, accu_val, _ = sess.run(
            [loss, accuracy, train_op], 
            feed_dict={
                x: batch_data,
                y: batch_labels})
        if i % 500 == 0:
            print('[Train] step: %d, loss: %4.5f, acc:%4.5f'% (i, loss_val, accu_val)) 
        if i % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            
            all_test_acc_val = []
            
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy],
                                       feed_dict={
                                           x: test_batch_data,
                                           y: test_batch_labels
                                       })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test] Step: %d, acc: %4.5f'%(i+1, test_acc)) 

[Train] step: 0, loss: 2.46483, acc:0.05000
(10000, 3072) (10000,)
[Test] Step: 1, acc: 0.09100
[Train] step: 500, loss: 1.78747, acc:0.40000
[Train] step: 1000, loss: 1.75127, acc:0.50000
[Train] step: 1500, loss: 1.03991, acc:0.60000
[Train] step: 2000, loss: 1.79623, acc:0.40000
[Train] step: 2500, loss: 1.36267, acc:0.50000
[Train] step: 3000, loss: 1.24740, acc:0.55000
[Train] step: 3500, loss: 1.40317, acc:0.40000
[Train] step: 4000, loss: 2.28772, acc:0.25000
[Train] step: 4500, loss: 1.82554, acc:0.40000
[Train] step: 5000, loss: 1.65145, acc:0.25000
(10000, 3072) (10000,)
[Test] Step: 5001, acc: 0.46350
[Train] step: 5500, loss: 0.74461, acc:0.80000
[Train] step: 6000, loss: 1.31147, acc:0.50000
[Train] step: 6500, loss: 1.24706, acc:0.60000
[Train] step: 7000, loss: 1.20062, acc:0.60000
[Train] step: 7500, loss: 1.29875, acc:0.50000
[Train] step: 8000, loss: 1.14909, acc:0.60000
[Train] step: 8500, loss: 1.60727, acc:0.45000
[Train] step: 9000, loss: 1.15952, acc:0.50000
[Tra